In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv


In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from io import StringIO
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib as mpl
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import openpyxl
import plotly.graph_objects as go
from matplotlib.sankey import Sankey
from xgboost import XGBRegressor
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [3]:
rain_train = "/kaggle/input/playground-series-s5e3/train.csv"
rain_test = "/kaggle/input/playground-series-s5e3/test.csv"

In [4]:
rain_train_df = pd.read_csv(rain_train)

In [5]:
rain_test_df = pd.read_csv(rain_test)

# Outline

Objective: Create a deep learning model to predict rainfall via machine learning.

How: Model building. 

Evaluation: Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target. 

Prediction Target: rainfall column in rain_train. 

In [6]:
for df in [rain_train_df, rain_test_df]:
    df['dewpoint_temparature_windspeed'] = df['windspeed'] * df['temparature'] * df['dewpoint']

In [7]:
max_sunshine = rain_train_df['sunshine'].max()
min_sunshine = rain_train_df['sunshine'].min()

for df in [rain_train_df, rain_test_df]:
    df['sunshine_normalized'] = (df['sunshine'] - min_sunshine) / (max_sunshine - min_sunshine)
    # Invert the sunshine effect (higher sunshine -> lower cloud cover)
    df['sunshine_cloud'] = 1 - df['sunshine_normalized']

# Modelling

In [8]:
feature_columns = ['windspeed', 'dewpoint', 'humidity', 'sunshine_cloud', 'temparature', 'dewpoint_temparature_windspeed']

In [9]:
# Split training data
X_train = rain_train_df[feature_columns]
y_train = rain_train_df['rainfall']

In [10]:
# Initialize and train the model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [11]:
X_test = rain_test_df[feature_columns]
rain_test_df['rainfall'] = xgb_model.predict(X_test)  # Add predictions to test DataFrame

In [12]:
submission = rain_test_df[['id', 'rainfall']]

# If ordered and un_ordered are required and available in test:
# submission['ordered'] = pastry_test_df['ordered']
# submission['un_ordered'] = pastry_test_df['un_ordered']

# Save submission file
submission.to_csv('submission.csv', index=False)
print("Submission file saved successfully!")

Submission file saved successfully!
